In [40]:
import ccxt
import pandas as pd
import numpy as np

# Initialize Binance
exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '1h'  # hourly candles
limit = 1000  # number of candles to fetch

# Fetch OHLCV data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# Convert to DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)


In [42]:
import talib

# RSI
df['RSI'] = talib.RSI(df['close'], timeperiod=14)

# MACD
df['MACD'], df['Signal'], df['Hist'] = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)

# DEMA
df['DEMA'] = talib.DEMA(df['close'], timeperiod=14)
df['DEMA_Signal'] = df['close'] - df['DEMA']

# SMA
df['SMA'] = talib.SMA(df['close'], timeperiod=14)
df['SMA_Signal'] = df['close'] - df['SMA']

# TSI (approximate using TEMA)
df['TSI'] = talib.TEMA(df['close'], timeperiod=25)

# Stochastic Oscillator
slowk, slowd = talib.STOCH(df['high'], df['low'], df['close'], 
                             fastk_period=14, slowk_period=3, slowk_matype=0,
                             slowd_period=3, slowd_matype=0)
df['%K'] = slowk
df['%D'] = slowd

# Fill missing values
df.fillna(method='ffill', inplace=True)
df.fillna(0, inplace=True)


/tmp/ipykernel_10237/3019899538.py:28: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [43]:
import numpy as np

lookback = 5  # number of candles before and after

def detect_swings(prices, highs, lows, lookback=5):
    labels = np.zeros(len(prices))  # default 0 = no reversal
    for i in range(lookback, len(prices) - lookback):
        # Swing High = downtrend reversal
        if highs[i] == max(highs[i-lookback:i+lookback+1]):
            labels[i] = -1
        # Swing Low = uptrend reversal
        elif lows[i] == min(lows[i-lookback:i+lookback+1]):
            labels[i] = 1
    return labels

# Apply to DataFrame
df['Label'] = detect_swings(df['close'], df['high'], df['low'], lookback=lookback)

# Drop initial/final rows where swing labels cannot be defined
df.dropna(inplace=True)

# Convert to 0,1,2 for PyTorch
df['Label'] = df['Label'] + 1  # -1 -> 0, 0 -> 1, 1 -> 2


/tmp/ipykernel_10237/1263483492.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if highs[i] == max(highs[i-lookback:i+lookback+1]):
/tmp/ipykernel_10237/1263483492.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif lows[i] == min(lows[i-lookback:i+lookback+1]):


In [45]:
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader

FEATURES = ["RSI", "MACD", "Signal", "Hist", "DEMA", "DEMA_Signal", 
            "SMA", "SMA_Signal", "TSI", "%K", "%D"]

X = df[FEATURES].values
y = torch.tensor(df['Label'].values, dtype=torch.long)

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Sliding window
window_size = 5
def create_windows(X, y, window_size):
    X_windows, y_windows = [], []
    for i in range(window_size, len(X)):
        X_windows.append(X[i-window_size:i])
        y_windows.append(y[i])
    return np.array(X_windows), np.array(y_windows)

X_windows, y_windows = create_windows(X, y.numpy(), window_size)

# Convert to tensors
X_windows = torch.tensor(X_windows, dtype=torch.float32)
y_windows = torch.tensor(y_windows, dtype=torch.long)

# Train/test split
split = int(0.8 * len(X_windows))
X_train, X_test = X_windows[:split], X_windows[split:]
y_train, y_test = y_windows[:split], y_windows[split:]

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [46]:
import torch.nn as nn

class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(CryptoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = out[:, -1, :]  # last time step
        out = self.fc(out)
        return out

input_dim = len(FEATURES)
hidden_dim = 64
num_layers = 2
output_dim = 3  # 0=downtrend, 1=no reversal, 2=uptrend

model = CryptoLSTM(input_dim, hidden_dim, num_layers, output_dim)


In [47]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 50

for epoch in range(epochs):
    model.train()
    running_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/50, Loss: 1.0559
Epoch 2/50, Loss: 0.8023
Epoch 3/50, Loss: 0.5291
Epoch 4/50, Loss: 0.4719
Epoch 5/50, Loss: 0.4584
Epoch 6/50, Loss: 0.4661
Epoch 7/50, Loss: 0.4678
Epoch 8/50, Loss: 0.4700
Epoch 9/50, Loss: 0.4620
Epoch 10/50, Loss: 0.4496
Epoch 11/50, Loss: 0.4577
Epoch 12/50, Loss: 0.4491
Epoch 13/50, Loss: 0.4468
Epoch 14/50, Loss: 0.4503
Epoch 15/50, Loss: 0.4268
Epoch 16/50, Loss: 0.4321
Epoch 17/50, Loss: 0.4314
Epoch 18/50, Loss: 0.4290
Epoch 19/50, Loss: 0.4184
Epoch 20/50, Loss: 0.4360
Epoch 21/50, Loss: 0.4317
Epoch 22/50, Loss: 0.4231
Epoch 23/50, Loss: 0.4284
Epoch 24/50, Loss: 0.4038
Epoch 25/50, Loss: 0.4205
Epoch 26/50, Loss: 0.4054
Epoch 27/50, Loss: 0.4025
Epoch 28/50, Loss: 0.4045
Epoch 29/50, Loss: 0.3928
Epoch 30/50, Loss: 0.3988
Epoch 31/50, Loss: 0.4078
Epoch 32/50, Loss: 0.3962
Epoch 33/50, Loss: 0.3976
Epoch 34/50, Loss: 0.3864
Epoch 35/50, Loss: 0.3848
Epoch 36/50, Loss: 0.3811
Epoch 37/50, Loss: 0.3789
Epoch 38/50, Loss: 0.3911
Epoch 39/50, Loss: 0.

In [48]:
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    predictions = torch.argmax(outputs, dim=1)

accuracy = (predictions == y_test).float().mean()
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.8894


In [49]:
torch.save(model.state_dict(), "crypto_reversal_model.pth")
print("Model saved successfully!")


Model saved successfully!
